In [5]:
import os
import pandas as pd
import numpy as np
import joblib

# To remove these warnings : "Your kernel may have been built without NUMA support."
#   run these 2 lines before importing tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'        # or any {'0', '1', '2', '3'}

import tensorflow as tf 
random_state = 10
np.random.seed(random_state)
tf.random.set_seed(random_state)

# store elements as dictionary keys and their counts as dictionary values
from collections import Counter

# scikit-learn
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

# Classification metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Function for creating model pipelines - imblearn
from imblearn.pipeline import make_pipeline as imbl_pipe

# Over-sampling using SMOTE
from imblearn.over_sampling import SMOTE

# SciKeras
from scikeras.wrappers import KerasClassifier

## Load Analytical Base Table

In [6]:
# Load the dataset
abt = pd.read_csv("../Resources/analytical_base_table.csv")
print(f"Dataframe dimensions: {abt.shape}")
abt.head()

Dataframe dimensions: (10000, 11)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# Models Training

Let's start by splitting our dataframe into separate objects:

* **y** for the target varibale

* **X** for the input features




### Separate dataframe into separate object

In [7]:
# Object for target variable
y = abt.Exited

# object for input features
X = abt.drop(['Exited'], axis=1)

# display shapes of X and y
print(X.shape, y.shape)

(10000, 10) (10000,)


In [8]:
# List numerical features
num_columns = X.select_dtypes(include='number').columns.tolist()
num_columns

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [9]:
# List categorical features
cat_columns = X.select_dtypes(include='object').columns.tolist()
cat_columns

['Geography', 'Gender']

In [10]:
def class_count(a):
    counter=Counter(a)
    kv=[list(counter.keys()),list(counter.values())]
    dff = pd.DataFrame(np.array(kv).T, columns=['Exited','Count'])
    dff['Count'] = dff['Count'].astype('int64')
    dff['%'] = round(dff['Count'] / a.shape[0] * 100, 2)
    return dff.sort_values('Count',ascending=False)

In [11]:
class_count(y)

,Exited,Count,%
1,0,7963,79.63
0,1,2037,20.37


## Create a Train Test Split


We will continue with splitting our data into separate training and test sets.

* 30% of observations will be set aside for the test set

* the rest, 70%, will be used as the training set

In [12]:
# Split X and y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=random_state,
                                                    stratify=abt.Exited)

# Print number of observations in X_train, X_test, y_train, and y_test
print(len(X_train), len(X_test), len(y_train), len(y_test))

7000 3000 7000 3000


In [13]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7000 entries, 8061 to 4741
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      7000 non-null   int64  
 1   Geography        7000 non-null   object 
 2   Gender           7000 non-null   object 
 3   Age              7000 non-null   int64  
 4   Tenure           7000 non-null   int64  
 5   Balance          7000 non-null   float64
 6   NumOfProducts    7000 non-null   int64  
 7   HasCrCard        7000 non-null   int64  
 8   IsActiveMember   7000 non-null   int64  
 9   EstimatedSalary  7000 non-null   float64
dtypes: float64(2), int64(6), object(2)
memory usage: 601.6+ KB


In [14]:
# Get a Numpy representation of the DataFrame
X_train = X_train.values
X_test = X_test.values

## Pre-processing Pipeline

### Scale numerical data and encode categorical data
Construct a pre-processing pipeline from the given transformers: MinMaxScaler and OneHotEncoder

Create lists of indexes from the list of column names

Need to be numeric not string to specify columns name in column transformer

In [15]:
num_features = [] 

for i in num_columns:
    location = X.columns.get_loc(i)
    num_features.append(location)
print(num_features)  

[0, 3, 4, 5, 6, 7, 8, 9]


In [16]:
cat_features = []

for i in cat_columns:
    location = X.columns.get_loc(i)
    cat_features.append(location)
print(cat_features)  

[1, 2]


In [18]:
# # Define column transformer
# # Need to be numeric not string to specify columns name 
# preprocess = make_column_transformer(
#     (MinMaxScaler(), num_features),
#     (OneHotEncoder(sparse=False), cat_features)
# )
# preprocess


from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler

# Define column transformer
preprocess = make_column_transformer(
    (MinMaxScaler(), num_features),
    (OneHotEncoder(sparse_output=False), cat_features)  # Use sparse_output instead of sparse
)

preprocess


ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                 [0, 3, 4, 5, 6, 7, 8, 9]),
                                ('onehotencoder',
                                 OneHotEncoder(sparse_output=False), [1, 2])])

## Create a Deep Learning Model

We are going to use SciKeras.  
It makes possible to use Keras/TensorFlow with sklearn by providing a wrapper around Keras that has an Scikit-Learn interface.

#### Implement the Scikit-Learn classifier interface.

In [19]:
# Import model build function "get_clf"from the script "keras_model.py"
import keras_model as km

In [20]:
from tensorflow.keras.metrics import Accuracy, Recall
from tensorflow.keras.losses import BinaryCrossentropy

In [21]:
clf = KerasClassifier(
    model=km.get_clf,
    loss=BinaryCrossentropy,
    metrics=[Accuracy, Recall],
    hidden_layer_sizes=(64, 32,),
    dropout=0.45,
    batch_size=64,
    optimizer='adam',
    optimizer__learning_rate=0.0021,
    epochs=8,
    verbose=0,
    random_state=random_state,  
)

### Build Model Pipeline with SMOTE

* We are going to use the Pipeline from the imblearn package in place of scikit-learn Pipeline.

* It takes care automatically to re-sample when called fit() on the pipeline, and does not re-sample test data (when called transform() or predict()).

In [22]:
# Define model with pipeline
pipe = imbl_pipe(preprocess,
                  SMOTE(sampling_strategy='auto', random_state=random_state),
                  clf)
pipe

AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('minmaxscaler',
                                                  MinMaxScaler(),
                                                  [0, 3, 4, 5, 6, 7, 8, 9]),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  [1, 2])])),
                ('smote', SMOTE(random_state=10)),
                ('kerasclassifier',
                 KerasClassifier(batch_size=64, dropout=0.45, epochs=8, hidden_layer_sizes=(64, 32), loss=<class 'keras.src.losses.losses.BinaryCrossentropy'>, metrics=[<class 'keras.src.metrics.accuracy_metrics.Accuracy'>, <class 'keras.src.metrics.confusion_metrics.Recall'>], model=<function get_clf at 0x00000244DC234AE0>, optimizer='adam', optimizer__learning_rate=0.0021, random_state=10, verbose=0))])

In [23]:
pipe.fit(X_train, y_train).score(X_test, y_test)

c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0.7726666666666666

In [24]:
pipe.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                    [0, 3, 4, 5, 6, 7, 8, 9]),
                                   ('onehotencoder',
                                    OneHotEncoder(sparse_output=False), [1, 2])])),
  ('smote', SMOTE(random_state=10)),
  ('kerasclassifier',
   KerasClassifier(
   	model=<function get_clf at 0x00000244DC234AE0>
   	build_fn=None
   	warm_start=False
   	random_state=10
   	optimizer=adam
   	loss=<class 'keras.src.losses.losses.BinaryCrossentropy'>
   	metrics=[<class 'keras.src.metrics.accuracy_metrics.Accuracy'>, <class 'keras.src.metrics.confusion_metrics.Recall'>]
   	batch_size=64
   	validation_batch_size=None
   	verbose=0
   	callbacks=None
   	validation_split=0.0
   	shuffle=True
   	run_eagerly=False
   	epochs=8
   	hidden_layer_sizes=(64, 32)
   	dropout=0.45
   	optimizer__learning_rate=0.0021
   	class_weight=None
   ))],
 'transform_inpu

## Hyperparameter Tuning

For hyperparameter tuning we will use sklearn's RandomizedSearch.

 - First we will define the search space.  
- This is a dictionary where names are arguments to the model and values are distributions from which to draw samples. 
- To optimize some hyperparameters we will use scipy distributions for random sampling.


In [25]:
# Distributions for random sampling
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

In [26]:
# Define search space
param_space = {
    'kerasclassifier__epochs': sp_randInt(8,12),
    'kerasclassifier__batch_size': [64, 128],
    'kerasclassifier__dropout': sp_randFloat(loc=0.4, scale=0.1),
    'kerasclassifier__hidden_layer_sizes': [(64,), (64, 32,)],
    'kerasclassifier__optimizer__learning_rate': sp_randFloat(loc=0.0015, scale=0.001)
}

In [27]:
# Use stratified cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

# Create the RandomizedSearchCV model 
rsearch = RandomizedSearchCV(pipe, 
                            param_space, 
                            n_iter=150, 
                            scoring='f1_weighted', 
                            n_jobs=4, 
                            cv=skf,
                            random_state=random_state)

#### Train the model with RandomizedSearch

In [28]:
# Train the model with RandomizedSearch
rsearch.fit(X_train, y_train)

c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=10, shuffle=True),
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('minmaxscaler',
                                                                               MinMaxScaler(),
                                                                               [0,
                                                                                3,
                                                                                4,
                                                                                5,
                                                                                6,
                                                                                7,
                                                                                8,
                                                                                9]),
                                                                              ('onehotencoder',
                                                                               OneHotEncoder(sparse_output=False),
                                                                               [1,
                                                                                2])])),
                                             ('smote', SMOTE(random_state=10)),
                                             ('kerasclassifier',
                                              KerasClassifier(batch_size=6...
                                        'kerasclassifier__dropout': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000244DBEA8770>,
                                        'kerasclassifier__epochs': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000244DBEA8680>,
                                        'kerasclassifier__hidden_layer_sizes': [(64,),
                                                                                (64,
                                                                                 32)],
                                        'kerasclassifier__optimizer__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000244DBEA6EA0>},
                   random_state=10, scoring='f1_weighted')

## Quantify our Trained Model¶

In [ ]:
# Summarize result
print(f"Best Score: {rsearch.best_score_}  using:\n{rsearch.best_params_}")

Best Score: 0.8006564760892573  using:
{'kerasclassifier__batch_size': 64, 'kerasclassifier__dropout': 0.4038293404185837, 'kerasclassifier__epochs': 9, 'kerasclassifier__hidden_layer_sizes': (64,), 'kerasclassifier__optimizer__learning_rate': 0.002332084761312749}


In [29]:
# Compare training and testing scores
print(f"Training Data Score: {rsearch.score(X_train, y_train)}")
print(f"Testing Data Score: {rsearch.score(X_test, y_test)}")

Training Data Score: 0.7677725077996184
Testing Data Score: 0.7504739135563612


## Make Predictions

In [30]:
# Make predictions with the hypertuned model
pred = rsearch.predict(X_test)
pred

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [31]:
accuracy_score(y_test, pred)

0.7253333333333334

#### Classification metrics

In [32]:
# Confusion matrix
cm = confusion_matrix(y_test, pred)
print(cm)

[[1690  699]
 [ 125  486]]


In [33]:
# Normalized confusion matrix
cm = np.around(cm / cm.sum(axis=1)[:, np.newaxis], 2)
print(cm)

[[0.71 0.29]
 [0.2  0.8 ]]


In [34]:
# Classification report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.93      0.71      0.80      2389
           1       0.41      0.80      0.54       611

    accuracy                           0.73      3000
   macro avg       0.67      0.75      0.67      3000
weighted avg       0.83      0.73      0.75      3000



#### Predictions

In [35]:
print(f"Predicted classes: \t{list(pred[:10])}")
print(f"Actual Labels: \t\t{list(y_test[:10])}")

Predicted classes: 	[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
Actual Labels: 		[1, 0, 0, 0, 0, 1, 0, 0, 0, 0]


In [36]:
pred1 = rsearch.predict(X_test[5:6])
print(f"Predicted classes: \t{list(pred1)}")
print(f"Actual Labels: \t\t{list(y_test[5:6])}")

Predicted classes: 	[1]
Actual Labels: 		[1]


#### Test for new data

In [37]:
X_new = X_test[0,:].reshape(1,-1)
X_new

array([[638, 'France', 'Male', 36, 6, 188455.19, 1, 0, 0, 47031.4]],
      dtype=object)

In [38]:
X_new.shape

(1, 10)

In [39]:
pred_new = rsearch.predict(X_new)

In [40]:
print(f"Predicted classes: \t{list(pred_new)}")
print(f"Actual Labels: \t\t{list(y_test[:1])}")

Predicted classes: 	[0]
Actual Labels: 		[1]


## Saving a Trained Model

In [41]:
file = '../models/scikeras.sav'
joblib.dump(rsearch, file)

['../models/scikeras.sav']

In [42]:
type(rsearch)

sklearn.model_selection._search.RandomizedSearchCV

## Loading a Saved Model

In [43]:
# Load the saved model
l_model = joblib.load(file)

## Evaluating the loaded model¶

In [44]:
print(l_model.score(X_test, y_test))

0.7504739135563612


In [45]:
# Make predictions with the hypertuned model
predl = l_model.predict(X_test)

In [46]:
accuracy_score(y_test, predl)

0.7253333333333334

In [47]:
cm = confusion_matrix(y_test, predl)
print(cm)

[[1690  699]
 [ 125  486]]


In [48]:
# Normalized confusion matrix
cm = np.around(cm / cm.sum(axis=1)[:, np.newaxis], 2)
print(cm)

[[0.71 0.29]
 [0.2  0.8 ]]


All looks good.

### Predict class for new data

In [49]:
# Let's use the first X_test record as new data
X_test[:1]

array([[638, 'France', 'Male', 36, 6, 188455.19, 1, 0, 0, 47031.4]],
      dtype=object)

In [50]:
predl_new = l_model.predict(X_test[:1])

In [51]:
print(f"Predicted classes: \t{list(predl_new)}")
print(f"Actual Labels: \t\t{list(y_test[:1])}")

Predicted classes: 	[0]
Actual Labels: 		[1]
